<a href="https://colab.research.google.com/github/knmehta12/Kitabe/blob/book_description_WIP/Book_desc.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import pandas as pd
import numpy as np
from nltk.corpus import stopwords
from sklearn.metrics.pairwise import linear_kernel
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.tokenize import RegexpTokenizer
import re
import string
import random
from PIL import Image
import requests
from io import BytesIO
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.metrics.pairwise import cosine_similarity
import nltk


In [ ]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [ ]:
desc = pd.read_csv('/content/drive/MyDrive/desc.csv')

In [ ]:
df=pd.read_csv('/content/drive/MyDrive/Books/books.csv')

In [ ]:
df['desc']=desc['desc']

In [ ]:
df.head()

,Unnamed: 0,r_index,book_id,authors,original_title,image_url,average_rating,ratings_count,genre,desc
0,0,1,2767052,Suzanne Collins,The Hunger Games,https://images.gr-assets.com/books/1447303603l...,4.34,4780653,"fiction, fantasy","In a future North America, where the rulers of..."
1,1,2,3,"J.K. Rowling, Mary GrandPré",Harry Potter and the Philosopher's Stone,https://images.gr-assets.com/books/1474154022l...,4.44,4602479,"fantasy, fiction",Rescued from the outrageous neglect of his aun...
2,2,3,41865,Stephenie Meyer,Twilight,https://images.gr-assets.com/books/1361039443l...,3.57,3866839,"fantasy, fiction",With 160 million copies of the Twilight Saga s...
3,3,4,2657,Harper Lee,To Kill a Mockingbird,https://images.gr-assets.com/books/1361975680l...,4.25,3198671,"classics, fiction",The unforgettable novel of a childhood in a sl...
4,4,5,4671,F. Scott Fitzgerald,The Great Gatsby,https://images.gr-assets.com/books/1490528560l...,3.89,2683664,"classics, fiction",A young man newly rich tries to recapture the ...


In [ ]:
df.tail()

,Unnamed: 0,r_index,book_id,authors,original_title,image_url,average_rating,ratings_count,genre,desc
9269,9269,9996,7130616,Ilona Andrews,Bayou Moon,https://images.gr-assets.com/books/1307445460l...,4.09,17204,"fantasy, romance","When her parents vanish, Cerise Mar becomes tr..."
9270,9270,9997,208324,Robert A. Caro,Means of Ascent,https://i.gr-assets.com/images/S/compressed.ph...,4.25,12582,"biography, history",Traces Johnson's life from his Texas childhood...
9271,9271,9998,77431,Patrick O'Brian,The Mauritius Command,https://images.gr-assets.com/books/1455373531l...,4.35,9421,"fiction, history","During the Napoleonic wars, British naval capt..."
9272,9272,9999,8565083,Peggy Orenstein,Cinderella Ate My Daughter: Dispatches from th...,https://images.gr-assets.com/books/1279214118l...,3.65,11279,"nonfiction, psychology",The acclaimed author of the groundbreaking bes...
9273,9273,10000,8914,John Keegan,The First World War,https://images.gr-assets.com/books/1403194704l...,4.00,9162,"history, nonfiction",A history of the Great War chronicles the even...


In [ ]:
def _removeNonAscii(s):
    return "".join(i for i in s if  ord(i)<128)
# Function for converting into lower case
def make_lower_case(text):
    return text.lower()
# Function for removing stop words
def remove_stop_words(text):
    text = text.split()
    stops = set(stopwords.words("english"))
    text = [w for w in text if not w in stops]
    text = " ".join(text)
    return text
# Function for removing punctuation
def remove_punctuation(text):
    tokenizer = RegexpTokenizer(r'\w+')
    text = tokenizer.tokenize(text)
    text = " ".join(text)
    return text
#Function for removing the html tags
def remove_html(text):
    html_pattern = re.compile('<.*?>')
    return html_pattern.sub(r'', text)

In [ ]:
df['cleaned_desc'] = df['desc'].apply(_removeNonAscii)
df['cleaned_desc'] = df.cleaned_desc.apply(func = make_lower_case)
df['cleaned_desc'] = df.cleaned_desc.apply(func = remove_stop_words)
df['cleaned_desc'] = df.cleaned_desc.apply(func=remove_punctuation)
df['cleaned_desc'] = df.cleaned_desc.apply(func=remove_html)


In [ ]:
df.drop(columns='desc')

,Unnamed: 0,r_index,book_id,authors,original_title,image_url,average_rating,ratings_count,genre,cleaned_desc
0,0,1,2767052,Suzanne Collins,The Hunger Games,https://images.gr-assets.com/books/1447303603l...,4.34,4780653,"fiction, fantasy",future north america rulers panem maintain con...
1,1,2,3,"J.K. Rowling, Mary GrandPré",Harry Potter and the Philosopher's Stone,https://images.gr-assets.com/books/1474154022l...,4.44,4602479,"fantasy, fiction",rescued outrageous neglect aunt uncle young bo...
2,2,3,41865,Stephenie Meyer,Twilight,https://images.gr-assets.com/books/1361039443l...,3.57,3866839,"fantasy, fiction",160 million copies twilight saga sold worldwid...
3,3,4,2657,Harper Lee,To Kill a Mockingbird,https://images.gr-assets.com/books/1361975680l...,4.25,3198671,"classics, fiction",unforgettable novel childhood sleepy southern ...
4,4,5,4671,F. Scott Fitzgerald,The Great Gatsby,https://images.gr-assets.com/books/1490528560l...,3.89,2683664,"classics, fiction",young man newly rich tries recapture past win ...
...,...,...,...,...,...,...,...,...,...,...
9269,9269,9996,7130616,Ilona Andrews,Bayou Moon,https://images.gr-assets.com/books/1307445460l...,4.09,17204,"fantasy, romance",parents vanish cerise mar becomes trapped war ...
9270,9270,9997,208324,Robert A. Caro,Means of Ascent,https://i.gr-assets.com/images/S/compressed.ph...,4.25,12582,"biography, history",traces johnson s life texas childhood rise pol...
9271,9271,9998,77431,Patrick O'Brian,The Mauritius Command,https://images.gr-assets.com/books/1455373531l...,4.35,9421,"fiction, history",napoleonic wars british naval captain jack aub...
9272,9272,9999,8565083,Peggy Orenstein,Cinderella Ate My Daughter: Dispatches from th...,https://images.gr-assets.com/books/1279214118l...,3.65,11279,"nonfiction, psychology",acclaimed author groundbreaking bestseller sch...


In [ ]:
df.to_csv('books.csv')

In [ ]:
df['cleaned_desc'].head()

0    future north america rulers panem maintain con...
1    rescued outrageous neglect aunt uncle young bo...
2    160 million copies twilight saga sold worldwid...
3    unforgettable novel childhood sleepy southern ...
4    young man newly rich tries recapture past win ...
Name: cleaned_desc, dtype: object

In [ ]:
np.savez("cleaned_desc.npz", df['cleaned_desc'])

In [ ]:
def create_soup(x):
    return   x['cleaned_desc'] + ' '+x['original_title'] + ' ' + x['average_rating']

In [ ]:
features = ['authors','original_title','cleaned_desc','average_rating']
df_filledbooks = df[features]
df_filledbooks.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9274 entries, 0 to 9273
Data columns (total 4 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   authors         9274 non-null   object 
 1   original_title  9274 non-null   object 
 2   cleaned_desc    9274 non-null   object 
 3   average_rating  9274 non-null   float64
dtypes: float64(1), object(3)
memory usage: 289.9+ KB


In [ ]:
df_filledbooks = df_filledbooks.astype(str)
df_filledbooks.dtypes

authors           object
original_title    object
cleaned_desc      object
average_rating    object
dtype: object

In [ ]:
df_filledbooks['soup'] = df_filledbooks.apply(create_soup, axis = 1)

In [ ]:
df_filledbooks['soup']

0       future north america rulers panem maintain con...
1       rescued outrageous neglect aunt uncle young bo...
2       160 million copies twilight saga sold worldwid...
3       unforgettable novel childhood sleepy southern ...
4       young man newly rich tries recapture past win ...
                              ...                        
9269    parents vanish cerise mar becomes trapped war ...
9270    traces johnson s life texas childhood rise pol...
9271    napoleonic wars british naval captain jack aub...
9272    acclaimed author groundbreaking bestseller sch...
9273    history great war chronicles events conflict e...
Name: soup, Length: 9274, dtype: object

In [ ]:

tfidf = TfidfVectorizer(stop_words = 'english',ngram_range=(1,2))
# m by n matrix
tf_matrix = tfidf.fit_transform(df_filledbooks['soup'])

cosine_sim2 = cosine_similarity(tf_matrix, tf_matrix)

In [ ]:
df_filledbooks = df_filledbooks.reset_index()
indices = pd.Series(data = df_filledbooks.index, index = df_filledbooks['original_title'])

In [ ]:
def get_recommendations_new(title, cosine_sim = cosine_sim2):
   
#   Get indice of given book
    idx = indices[title]
    
    # Get this books sim with all other books, enum to keep track of book index
    sim_scores = list(enumerate(cosine_sim[idx]))
    
    sim_scores = sorted(sim_scores, key = lambda x: x[1], reverse = True)
    # Except the book itself
    sim_scores = sim_scores[1:11]
    
    movie_indices  = [i[0] for i in sim_scores]
    
    return list(df['original_title'].iloc[movie_indices])

In [ ]:
 get_recommendations_new("The Hunger Games")

['The Hunger Games Box Set',
 'Mockingjay',
 'Catching Fire',
 'Legend',
 'Forbidden',
 'Hunger ',
 'Partials',
 'The Knight at Dawn (Magic Tree House, #2)',
 'How to Talk to a Widower',
 'The Long Walk']

In [ ]:
get_recommendations_new("Twilight")

['Twilight: The Graphic Novel,',
 'The Twilight Collection (Twilight, #1-3)',
 'Midnight Sun (Partial Draft)',
 'Twilight and History',
 'The Twilight Saga: The Official Guide',
 'The Twilight Saga',
 'Eclipse',
 'New Moon (Twilight, #2)',
 'Holding Up the Universe',
 'Twilight: The Graphic Novel']

In [ ]:
get_recommendations_new("بين القصرين")

['أولاد حارتنا',
 'Olive Kitteridge',
 '沈黙',
 'Two for the Dough',
 'Junky',
 'The Map That Changed the World: William Smith and the Birth of Modern Geology',
 'Dreamcatcher',
 "The Children's Book",
 'Le cosmicomiche',
 'Size 14 Is Not Fat Either']

In [ ]:
get_recommendations_new("東京喰種トーキョーグール 1")

['Lothaire',
 'النبطي',
 'رباعيات صلاح جاهين',
 'Dreamfever',
 'Zoo',
 'Dead Beat',
 'Magic Slays',
 'Motorcycle Man',
 'The Undead Pool',
 'This Man Confessed']

In [ ]:
np.savez_compressed('cosine_rating_sim.npz', array1=cosine_sim2)